In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from IPython import display as ipd
import soundfile as sf
from tqdm import tqdm
from libsoni.core.pianoroll import sonify_pianoroll_additive_synthesis, \
                                   sonify_pianoroll_fm_synthesis, \
                                   sonify_pianoroll_clicks, \
                                   sonify_pianoroll_sample, \
                                   visualize_pianoroll

from libsoni.core.tse import sonify_tse_clicks
from libsoni.util.utils import mix_sonification_and_original, get_preset

DATASET_DIR = '/media/yiitozer/Elements/pc-pipeline/datasets/20230915_separation_transcription_dataset/train/'


Fs=22050

In [ ]:
for item_idx in tqdm(sorted(os.listdir(DATASET_DIR))):
    if os.path.isfile(os.path.join(DATASET_DIR, item_idx, 'piano_sonified.wav')):
        continue
    
    try:
        pc_df = pd.read_csv(os.path.join(DATASET_DIR, item_idx, 'piano.csv'),sep=';')
    except:
        continue
    pc_audio, _ = librosa.load(os.path.join(DATASET_DIR, item_idx, 'piano.wav'), sr=Fs)
    fm = sonify_pianoroll_additive_synthesis(pianoroll_df=pc_df,
                                             partials=[1, 2, 3],
                                             partials_amplitudes=[0.5, 0.25, 0.25])
    clicks = sonify_pianoroll_clicks(pianoroll_df=pc_df)
    soni = fm + clicks * 10

    sonification_stereo = mix_sonification_and_original(soni, pc_audio, gain_lin_sonification=10)
    sonification_stereo /= sonification_stereo.max()

    sf.write(os.path.join(DATASET_DIR, item_idx, 'piano_sonified.wav'), sonification_stereo.T, Fs)

In [ ]:
ipd.Audio(sonification_stereo, rate=Fs)